In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

# Define the dataset directory
dataset_dir = r'C:\Users\Tharanesh P\OneDrive\Desktop\FINAL_CODE_TRAIN\FINAL_CODE_TRAIN\DATASET\colab_data'  # Update this to your dataset path

# Custom Dataset class
class AgeDataset(Dataset):
    def __init__(self, dataset_dir):
        self.images = []
        self.labels = []
        self.age_categories = ['Age-1', 'Age-2', 'Age-3', 'Age-4', 'Age-5', 
                               'Age-6', 'Age-7', 'Age-8', 'Age-9', 'Age-10']

        # Load images and labels
        for age_index, category in enumerate(self.age_categories):
            category_path = os.path.join(dataset_dir, category)
            for filename in os.listdir(category_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    img_path = os.path.join(category_path, filename)
                    self.images.append(img_path)
                    self.labels.append(age_index)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        transform = transforms.Compose([
            transforms.Resize((200, 200)),  # Resize to 200x200
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        image = transform(image)
        label = self.labels[idx]
        return image, label

# Define a simple GAN architecture (for example purposes)
class SimpleGAN(nn.Module):
    def __init__(self):
        super(SimpleGAN, self).__init__()
        # Define your generator and discriminator networks
        self.generator = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 3 * 200 * 200),  # Output size for 200x200 RGB image
            nn.Tanh()
        )

        self.discriminator = nn.Sequential(
            nn.Linear(3 * 200 * 200, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),  # Output a single value for binary classification
            nn.Sigmoid()
        )

    def generate(self, noise):
        return self.generator(noise)

    def discriminate(self, image):
        return self.discriminator(image.view(image.size(0), -1))

# Training Function
def train_gan(dataset_dir, num_epochs=100, batch_size=32):
    dataset = AgeDataset(dataset_dir)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = SimpleGAN()
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(model.generator.parameters(), lr=0.0002)
    optimizer_d = optim.Adam(model.discriminator.parameters(), lr=0.0002)

    for epoch in range(num_epochs):
        for real_images, _ in dataloader:
            batch_size = real_images.size(0)
            noise = torch.randn(batch_size, 100)

            # Train Discriminator
            optimizer_d.zero_grad()
            real_labels = torch.ones(batch_size, 1)
            fake_images = model.generate(noise)
            real_loss = criterion(model.discriminate(real_images), real_labels)
            fake_labels = torch.zeros(batch_size, 1)
            fake_loss = criterion(model.discriminate(fake_images.detach()), fake_labels)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            g_loss = criterion(model.discriminate(fake_images), real_labels)
            g_loss.backward()
            optimizer_g.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')

    # Save the trained model
    torch.save(model.state_dict(), 'gan_model.pth')

# Run the training
train_gan(dataset_dir= r'C:\Users\Tharanesh P\OneDrive\Desktop\FINAL_CODE_TRAIN\FINAL_CODE_TRAIN\DATASET\colab_data', num_epochs=100, batch_size=32)  # Adjust parameters as needed


Epoch [1/100], D Loss: 0.0835, G Loss: 4.0417
Epoch [2/100], D Loss: 0.0118, G Loss: 6.0795
Epoch [3/100], D Loss: 0.0003, G Loss: 11.7666
Epoch [4/100], D Loss: 0.0550, G Loss: 3.8769
Epoch [5/100], D Loss: 0.3275, G Loss: 4.7045
Epoch [6/100], D Loss: 0.0032, G Loss: 8.5881
Epoch [7/100], D Loss: 0.0014, G Loss: 9.2373
Epoch [8/100], D Loss: 0.1356, G Loss: 5.5513
Epoch [9/100], D Loss: 2.0094, G Loss: 0.8519
Epoch [10/100], D Loss: 0.8099, G Loss: 0.5859
Epoch [11/100], D Loss: 0.2481, G Loss: 7.5330
Epoch [12/100], D Loss: 0.0002, G Loss: 13.5138
Epoch [13/100], D Loss: 0.0000, G Loss: 96.0171
Epoch [14/100], D Loss: 0.0000, G Loss: 58.0449
Epoch [15/100], D Loss: 0.0000, G Loss: 67.6937
Epoch [16/100], D Loss: 0.0000, G Loss: 59.8371
Epoch [17/100], D Loss: 0.0020, G Loss: 15.2690
Epoch [18/100], D Loss: 0.0004, G Loss: 23.6106
Epoch [19/100], D Loss: 0.0000, G Loss: 93.4509
Epoch [20/100], D Loss: 0.0000, G Loss: 71.1549
Epoch [21/100], D Loss: 0.0000, G Loss: 77.3244
Epoch [22/1

In [1]:
import os
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms

# Define the SimpleGAN class with generator and discriminator
class SimpleGAN(nn.Module):
    def __init__(self):
        super(SimpleGAN, self).__init__()
        # Define the generator
        self.generator = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 3 * 200 * 200),  # Output size for 200x200 RGB image
            nn.Tanh()  # Outputs values in the range [-1, 1]
        )

        # Define the discriminator
        self.discriminator = nn.Sequential(
            nn.Linear(3 * 200 * 200, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),  # Output a single value for binary classification
            nn.Sigmoid()
        )

    def generate(self, noise):
        output = self.generator(noise)  # Output shape should be [1, 3*200*200]
        output = output.view(-1, 3, 200, 200)  # Reshape to (1, 3, 200, 200)
        return output

# Function to load the trained model
def load_model(model_path):
    model = SimpleGAN()  # Create an instance of your model
    model.load_state_dict(torch.load(model_path))  # Load the state dict
    model.eval()  # Set the model to evaluation mode
    return model

# Function to generate age-transformed images and save them as a GIF
def generate_age_gif(model, input_image_path, start_age, end_age, output_gif_path):
    image = Image.open(input_image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((200, 200)),  # Resize to 200x200
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    frames = []  # List to store generated frames for the GIF

    with torch.no_grad():  # No gradient calculation needed for inference
        for age in range(start_age, end_age + 1):
            noise = torch.randn(1, 100)  # Generate random noise for each age step
            generated_image = model.generate(noise)  # Generate image

            # Ensure generated_image has the right shape before permuting
            if generated_image.dim() == 4:  # Check if it's [batch_size, channels, height, width]
                generated_image = generated_image.squeeze(0)  # Remove batch dimension
                generated_image = generated_image.permute(1, 2, 0)  # Change shape to (H, W, C)
            else:
                raise ValueError(f"Unexpected output shape from the generator: {generated_image.shape}")

            generated_image = (generated_image * 0.5 + 0.5) * 255  # Rescale to [0, 255]
            generated_image = Image.fromarray(generated_image.numpy().astype('uint8'))  # Convert to PIL Image
            frames.append(generated_image)  # Append to frames list

    # Save frames as a GIF
    frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=300, loop=0)
    print(f"GIF saved at: {output_gif_path}")

# Example usage
if __name__ == '__main__':
    model_path = r'C:\Users\Tharanesh P\OneDrive\Desktop\FINAL_CODE_TRAIN\FINAL_CODE_TRAIN\model\gan_model.pth'  # Update this to your trained model path
    input_image_path = r'C:\Users\Tharanesh P\OneDrive\Desktop\FINAL_CODE_TRAIN\FINAL_CODE_TRAIN\DATASET\colab_data\Age-2\2.jpg'  # Update this to your input image path
    start_age = 1  # Starting age for GIF
    end_age = 10  # Ending age for GIF
    output_gif_path = 'age_transformation.gif'  # Path to save the GIF

    # Load the trained model
    model = load_model(model_path)

    # Generate and save the age transformation GIF
    generate_age_gif(model, input_image_path, start_age, end_age, output_gif_path)


C:\Users\Tharanesh P\AppData\Local\Temp\ipykernel_3300\3862802098.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load 

GIF saved at: age_transformation.gif


In [2]:
import os
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms

# Define the SimpleGAN class with generator and discriminator
class SimpleGAN(nn.Module):
    def __init__(self):
        super(SimpleGAN, self).__init__()
        # Define the generator
        self.generator = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 3 * 200 * 200),  # Output size for 200x200 RGB image
            nn.Tanh()  # Outputs values in the range [-1, 1]
        )

        # Define the discriminator
        self.discriminator = nn.Sequential(
            nn.Linear(3 * 200 * 200, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),  # Output a single value for binary classification
            nn.Sigmoid()
        )

    def generate(self, noise):
        output = self.generator(noise)  # Output shape should be [1, 3*200*200]
        output = output.view(-1, 3, 200, 200)  # Reshape to (1, 3, 200, 200)
        return output

# Function to load the trained model
def load_model(model_path):
    model = SimpleGAN()  # Create an instance of your model
    model.load_state_dict(torch.load(model_path))  # Load the state dict
    model.eval()  # Set the model to evaluation mode
    return model

# Function to generate an age-transformed image
def generate_age_image(model, input_image_path, target_age):
    image = Image.open(input_image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((200, 200)),  # Resize to 200x200
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():  # No gradient calculation needed for inference
        noise = torch.randn(1, 100)  # Generate random noise
        generated_image = model.generate(noise)  # Generate image

    # Ensure generated_image has the right shape before permuting
    if generated_image.dim() == 4:  # Check if it's [batch_size, channels, height, width]
        generated_image = generated_image.squeeze(0)  # Remove batch dimension
        generated_image = generated_image.permute(1, 2, 0)  # Change shape to (H, W, C)
    else:
        raise ValueError(f"Unexpected output shape from the generator: {generated_image.shape}")

    generated_image = (generated_image * 0.5 + 0.5) * 255  # Rescale to [0, 255]
    generated_image = Image.fromarray(generated_image.numpy().astype('uint8'))  # Convert to PIL Image

    return generated_image

# Example usage
if __name__ == '__main__':
    model_path = 'gan_model.pth'  # Update this to your trained model path
    input_image_path = r'C:\Users\Tharanesh P\OneDrive\Desktop\FINAL_CODE_TRAIN\FINAL_CODE_TRAIN\DATASET\colab_data\Age-10\test1.png'  # Update this to your input image path
    target_age = 10  # Example target age (1 to 10)

    # Load the trained model
    model = load_model(model_path)

    # Generate and display the image
    generated_image = generate_age_image(model, input_image_path, target_age)
    generated_image.show()  # Display the generated image
    generated_image.save('generated_image3.jpg')  # Save the generated image


C:\Users\Tharanesh P\AppData\Local\Temp\ipykernel_3300\4219884977.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load 